# Phase Diagram: Péclet Number in $(c, K)$ Parameter Space

**A publishable figure missing from Paper 4C: the drift/diffusion phase boundary.**

The THRML Ising model predicts that the Péclet number depends on two parameters:
the constraint level $c$ (platform-specific, measured from behavioral data)
and the spin count $K$ (hardware-specific, set by TSU resolution).
Sweeping the full $(c, K)$ space reveals a critical line where $\text{Pe} = 1$
— the boundary between drift-dominated and diffusion-dominated dynamics.

This notebook generates the contour map, derives the critical line analytically,
and overlays the three empirical chain measurements from EXP-021.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
from scipy.special import expit  # sigmoid

**Analytic Pe formula**

From the mean-field Ising drift cascade (derived in `07_pe_calibration.ipynb`):
$$
\text{Pe}(b_\alpha, b_\gamma, c, K)
= K \cdot \sinh\!\bigl(2(b_\alpha - c\,b_\gamma)\bigr).
$$

Setting $\text{Pe} = 1$ and solving for $c$ gives the critical line:
$$
c_{\text{crit}}(K) = \frac{b_\alpha - \tfrac{1}{2}\operatorname{arcsinh}(1/K)}{b_\gamma}.
$$

For large $K$, $\operatorname{arcsinh}(1/K) \approx 1/K \to 0$, so
$c_{\text{crit}} \to b_\alpha / b_\gamma$ (the neutral-bias asymptote).
For small $K$, the critical $c$ is lower — less constraint is needed to
suppress drift when the spin count is small.

In [ ]:
# Canonical THRML parameters (Langevin EXP-001, Paper 4)
b_alpha = 0.5 * np.log(0.85 / 0.15)            # ≈ 0.867
b_gamma = b_alpha - 0.5 * np.log(0.06 / 0.94)  # ≈ 2.244

print(f"Parameters: b_alpha = {b_alpha:.4f}, b_gamma = {b_gamma:.4f}")
print(f"Neutral-bias asymptote: b_alpha/b_gamma = {b_alpha/b_gamma:.4f}")


def pe_analytic(c, K, b_alpha=b_alpha, b_gamma=b_gamma):
    """Pe = K * sinh(2 * (b_alpha - c * b_gamma))."""
    b_net = b_alpha - c * b_gamma
    return K * np.sinh(2 * b_net)


def c_crit(K, b_alpha=b_alpha, b_gamma=b_gamma):
    """Critical constraint c where Pe = 1: c_crit = (b_alpha - arcsinh(1/K)/2) / b_gamma."""
    return (b_alpha - np.arcsinh(1.0 / K) / 2.0) / b_gamma


# Sample critical line values
K_check = [1, 2, 4, 8, 16, 32, 64, 128, 1024]
print(f"\nCritical line c_crit(K):")
print(f"{'K':>6}  {'c_crit':>8}  {'Pe at c=0':>12}")
print("-" * 32)
for K_val in K_check:
    cc = float(c_crit(K_val))
    pe0 = float(pe_analytic(0.0, K_val))
    print(f"{K_val:>6}  {cc:>8.4f}  {pe0:>12.2f}")

In [ ]:
# (c, K) grid — full sweep
c_vals = np.linspace(0.0, 0.95, 400)
K_vals = np.logspace(0, 6, 400)  # K from 1 to 1e6
C_mesh, K_mesh = np.meshgrid(c_vals, K_vals)

Pe_mesh = pe_analytic(C_mesh, K_mesh)
# Clamp to positive (some (c,K) combinations give negative Pe if c*b_gamma > b_alpha)
Pe_mesh_pos = np.where(Pe_mesh > 0, Pe_mesh, 1e-6)
log_Pe = np.log10(Pe_mesh_pos)

# Critical line
K_crit_range = np.logspace(0, 6, 1000)
c_crit_vals = c_crit(K_crit_range)
# Clip to valid range [0, 1]
valid_mask = (c_crit_vals >= 0) & (c_crit_vals <= 1.0)

print(f"Pe range in grid: [{Pe_mesh_pos.min():.2e}, {Pe_mesh_pos.max():.2e}]")
print(f"log10(Pe) range: [{log_Pe.min():.2f}, {log_Pe.max():.2f}]")

In [ ]:
# EXP-021 empirical Pe values at K=16
empirical = [
    {"name": "ETH",  "pe": 3.74,  "color": "gold",     "marker": "o"},
    {"name": "SOL",  "pe": 16.17, "color": "limegreen", "marker": "s"},
    {"name": "DEG",  "pe": 25.5,  "color": "tomato",    "marker": "^"},
]
K_data = 16

def c_from_pe(pe_target, K=K_data):
    b_net = np.arcsinh(pe_target / K) / 2
    return (b_alpha - b_net) / b_gamma

for pt in empirical:
    pt["c"] = float(c_from_pe(pt["pe"]))
    print(f"{pt['name']}: c = {pt['c']:.4f}, Pe = {pt['pe']}")

In [ ]:
# Phase diagram
fig, ax = plt.subplots(figsize=(10, 7))

# Pe contour fill — log scale
vmin, vmax = -1.0, 5.0  # log10(Pe) from 0.1 to 1e5
cf = ax.contourf(
    C_mesh, K_mesh, log_Pe,
    levels=np.linspace(vmin, vmax, 60),
    cmap="RdYlBu_r",
    extend="both",
)
cbar = plt.colorbar(cf, ax=ax, fraction=0.046, pad=0.04)
cbar.set_label(r"$\log_{10}$(Pe)", fontsize=12)
cbar.set_ticks(range(-1, 6))
cbar.set_ticklabels([f"$10^{{{i}}}$" for i in range(-1, 6)])

# Pe = 1 critical line
ax.plot(
    c_crit_vals[valid_mask], K_crit_range[valid_mask],
    "k-", linewidth=2.5, label=r"Pe = 1 (drift/diffusion boundary)",
)

# Pe contour lines (selected)
for pe_level in [0.1, 1, 10, 100, 1000]:
    ax.contour(
        C_mesh, K_mesh, Pe_mesh_pos,
        levels=[pe_level], colors=["white"], linewidths=[0.6], alpha=0.4,
    )

# Empirical data points at K=16
for pt in empirical:
    ax.scatter(
        pt["c"], K_data,
        s=200, marker=pt["marker"], color=pt["color"],
        edgecolors="black", linewidth=1.5, zorder=10,
        label=f"{pt['name']}  Pe={pt['pe']}  c={pt['c']:.3f}",
    )

# K=16 hardware line
ax.axhline(y=K_data, color="white", linestyle="--", linewidth=1.2, alpha=0.6, label=f"K = {K_data} (THRML default)")

# Annotations
ax.text(0.05, 1e4, "Drift-dominated\n(Pe > 1)",
        fontsize=12, color="white", ha="left", va="center", fontweight="bold")
ax.text(0.70, 1.5, "Diffusion-dominated\n(Pe < 1)",
        fontsize=11, color="black", ha="left", va="center", fontweight="bold")

ax.set_xlabel("Constraint level $c$", fontsize=13)
ax.set_ylabel("Spins per agent $K$", fontsize=13)
ax.set_yscale("log")
ax.set_xlim(0.0, 0.95)
ax.set_ylim(1, K_vals.max())
ax.set_title(
    r"Péclet number phase diagram — $(c, K)$ space"
    f"\n(canonical $b_\\alpha$={b_alpha:.3f}, $b_\\gamma$={b_gamma:.3f})",
    fontsize=13,
)
ax.legend(loc="upper right", fontsize=9, framealpha=0.85)
plt.tight_layout()
plt.show()

In [ ]:
# Critical line: analytic formula and limiting behavior
K_dense = np.logspace(0, 6, 2000)
c_crit_dense = c_crit(K_dense)
asymptote = b_alpha / b_gamma  # c_crit as K -> inf

fig, ax = plt.subplots(figsize=(8, 5))
ax.semilogx(K_dense, c_crit_dense, "k-", linewidth=2, label=r"$c_{\rm crit}(K)$")
ax.axhline(y=asymptote, color="gray", linestyle="--", alpha=0.6,
           label=rf"Asymptote $b_\alpha/b_\gamma = {asymptote:.3f}$")

# Mark K=16 and the three chain positions
cc_16 = float(c_crit(K_data))
ax.axvline(x=K_data, color="steelblue", linestyle=":", alpha=0.5)
ax.scatter([K_data], [cc_16], s=120, color="steelblue", zorder=5,
           label=f"K=16: $c_{{\\rm crit}}$ = {cc_16:.3f}")

for pt in empirical:
    ax.scatter([K_data], [pt["c"]], s=100, marker=pt["marker"], color=pt["color"],
               edgecolors="black", linewidth=1, zorder=10,
               label=f"{pt['name']} c={pt['c']:.3f} (Pe={pt['pe']})")

# Shade drift-dominated region below the critical line
ax.fill_between(K_dense, 0, c_crit_dense, alpha=0.08, color="red", label="Drift-dominated (Pe > 1)")
ax.fill_between(K_dense, c_crit_dense, 1.0, alpha=0.08, color="blue", label="Diffusion-dominated (Pe < 1)")

ax.set_xlabel("Spins per agent $K$", fontsize=12)
ax.set_ylabel("Constraint level $c$", fontsize=12)
ax.set_title(r"Critical line $c_{\rm crit}(K)$: Pe = 1 boundary", fontsize=12)
ax.set_xlim(1, K_dense.max())
ax.set_ylim(0.0, 0.55)
ax.legend(fontsize=9, loc="lower right")
plt.tight_layout()
plt.show()

print(f"Critical line at K={K_data}: c_crit = {cc_16:.4f}")
for pt in empirical:
    margin = cc_16 - pt["c"]
    print(f"  {pt['name']}: c = {pt['c']:.4f}  =>  margin to boundary = {margin:.4f}")

In [ ]:
# Quantitative summary: Pe at each grid corner + empirical positions
print("Pe at grid extremes (K=16):")
print(f"  c=0.00 (unconstrained): Pe = {pe_analytic(0.00, 16):.1f}")
print(f"  c=0.10 (DEG region):    Pe = {pe_analytic(0.10, 16):.1f}")
print(f"  c=0.19 (SOL region):    Pe = {pe_analytic(0.19, 16):.1f}")
print(f"  c=0.33 (ETH region):    Pe = {pe_analytic(0.33, 16):.1f}")
print(f"  c={cc_16:.3f} (critical):  Pe = {pe_analytic(cc_16, 16):.3f}  (expected: 1.000)")
print(f"  c=0.50 (over-constrained): Pe = {pe_analytic(0.50, 16):.3f}")

print(f"\nPe asymptotic behavior as K → ∞ (at c=0.33):")
for K_val in [16, 64, 256, 1024, 4096]:
    print(f"  K={K_val:>5}: Pe = {pe_analytic(0.33, K_val):.1f}")

**Discussion**

The $(c, K)$ phase diagram reveals the structure of the drift/diffusion boundary in the THRML model.

**Critical line behavior.** $c_{\rm crit}(K)$ increases monotonically from $0.190$ at $K=1$
to the asymptote $b_\alpha/b_\gamma \approx 0.386$ as $K \to \infty$.
Higher spin count amplifies both drift and diffusion, but drift wins:
$\text{Pe} = K \sinh(2b)$ grows linearly in $K$ for fixed $b > 0$.
This means TSU hardware with larger $K$ is *harder* to push into the diffusion-dominated regime —
a design consideration for constraint architecture.

**Empirical positions.** All three EXP-021 chains sit in the drift-dominated region at $K=16$:

| Chain | $c$ | Pe | Margin to boundary |
|-------|-----|----|--------------------|
| ETH   | 0.335 | 3.74 | 0.038 (closest) |
| SOL   | 0.187 | 16.2 | 0.186 |
| DEG   | 0.108 | 25.5 | 0.265 |

ETH is closest to the boundary: a constraint increase of $\Delta c \approx 0.038$
would push it across to $\text{Pe} < 1$. This is a falsifiable prediction —
a regulatory intervention that increases $c$ for Ethereum trading by roughly $10\%$
relative to its current value should suppress drift below the diffusion threshold.

**Missing figure.** This contour is the phase diagram referenced but not generated
in Paper 4C §5 (Demon Lattice Phases). The Gas/Fluid/Crystal/Vortex boundary in that
paper maps to the Pe $= 1$ critical line here, with the $(c, K)$ parameterization
making the hardware-platform relationship explicit.

**Relates to:** `07_pe_calibration.ipynb` (empirical calibration); Paper 4C §5; EXP-021.